# Text generation using a RNN

Given a sequence of words from this data, train a model to predict the next word in the sequence. Longer sequences of text can be generated by calling the model repeatedly.

**Mount your Google Drive**

In [29]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [0]:
import os
os.chdir('/content/drive/My Drive/Labs/Residency 9/External Lab/')

### Import Keras and other libraries

In [0]:
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.layers import LSTM
from keras  import callbacks
from keras import optimizers
import pandas as pd 
import tensorflow as tf
import numpy as np

## Download data
Reference: Data is collected from http://www.gutenberg.org

For the lab purpose, you can load the dataset provided by Great Learning

### Load the Oscar Wilde dataset

Store all the ".txt" file names in a list

In [0]:
#project_path = "/content/drive/My Drive/Labs/Residency 9/External Lab/"

In [0]:
#from zipfile import ZipFile
#with ZipFile(project_path+'train.zip', 'r') as z:
 # z.extractall()

In [0]:
#from zipfile import ZipFile
#with ZipFile(project_path+'data.zip', 'r') as z:
#  z.extractall()

In [0]:
import glob
shakelist = glob.glob("./data/*.txt")

In [36]:
shakelist

['./data/For Love of the King.txt',
 './data/The Canterville Ghost.txt',
 './data/Salomé A tragedy in one act.txt',
 './data/Miscellaneous Aphorisms_ The Soul of Man.txt',
 './data/Rose Leaf and Apple Leaf.txt',
 './data/Lord Arthur Savile_s Crime.txt',
 './data/Essays and Lectures.txt',
 './data/Charmides and Other Poems.txt',
 './data/Vera or, The Nihilists.txt',
 './data/The Happy Prince and other tales.txt',
 './data/Poems with the Ballad of Reading Gaol.txt',
 './data/A Woman of No Importance a play.txt',
 './data/An Ideal Husband.txt',
 './data/Oscar Wilde Miscellaneous.txt',
 './data/Selected poems of oscar wilde including The Ballad of Reading Gaol.txt',
 './data/The Duchess of Padua.txt',
 './data/Children in Prison and Other Cruelties of Prison Life.txt',
 './data/Shorter Prose Pieces.txt',
 './data/Reviews.txt',
 './data/The Ballad of Reading Gaol.txt',
 './data/Miscellanies.txt',
 './data/De Profundis.txt',
 './data/A Critic in Pall Mall.txt',
 './data/Impressions of Ameri

### Read the data

Read contents of every file from the list and append the text in a new list

In [0]:
codetext = []
bookranges = []
for shakefile in shakelist:
    shaketext = open(shakefile, "r")
    start = len(codetext)
    codetext.append(shaketext.read())
    end = len(codetext)
    bookranges.append({"start": start, "end": end, "name": shakefile.rsplit("/", 1)[-1]})
    shaketext.close()

## Process the text
Initialize and fit the tokenizer

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(codetext)

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping words to numbers, and another for numbers to words.

In [0]:
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word

Get the word count for every word and also get the total number of words.

In [0]:
word_counts = tokenizer.word_counts
num_words = len(word_idx) + 1

Convert text to sequence of numbers

In [0]:
sequences = tokenizer.texts_to_sequences(codetext)

### Generate Features and Labels

In [0]:
features = []
labels = []

training_length = 50
# Iterate through the sequences of tokens
for seq in sequences:
    # Create multiple training examples from each sequence
    for i in range(training_length, training_length+300):
        # Extract the features and label
        extract = seq[i - training_length: i - training_length + 20]

        # Set the features and label
        features.append(extract[:-1])
        labels.append(extract[-1])

### The prediction task

Given a word, or a sequence of words, what is the most probable next word? This is the task we're training the model to perform. The input to the model will be a sequence of words, and we train the model to predict the output—the following word at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the words computed until this moment, what is the next word?

### Generate training and testing data

In [0]:
from sklearn.utils import shuffle
import numpy as np

features, labels = shuffle(features, labels, random_state=1)

# Decide on number of samples for training
train_end = int(0.7 * len(labels))

train_features = np.array(features[:train_end])
valid_features = np.array(features[train_end:])

train_labels = labels[:train_end]
valid_labels = labels[train_end:]

# Convert to arrays
X_train, X_valid = np.array(train_features), np.array(valid_features)

# Using int8 for memory savings
y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)
y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

# One hot encoding of labels
for example_index, word_index in enumerate(train_labels):
    y_train[example_index, word_index] = 1

for example_index, word_index in enumerate(valid_labels):
    y_valid[example_index, word_index] = 1

This is just to check the features and labels

In [44]:
for i, sequence in enumerate(X_train[:2]):
    text = []
#     print(i, sequence)
    for idx in sequence:
        text.append(idx_word[idx])
        
    print('Features: ' + ' '.join(text)+'\n')
    print('Label: ' + idx_word[np.argmax(y_train[i])] + '\n')

Features: net 1916 fourteenth and fifteenth edition 1 s net 1917 sixteenth edition 5 s net 1917 the literary and

Label: dramatic

Features: project gutenberg ebook essays and lectures transcribed from the 1913 methuen and co edition by david price email ccx074

Label: pglaf



## Build The Model

import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backendUse `keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `keras.layers.LSTM`: A type of RNN with size `units=rnn_units` (You can also use a GRU layer here.)
* `keras.layers.Dense`: The output layer, with `num_words` outputs.

In [0]:
import glob

from sklearn.utils import shuffle
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.optimizers import Adam
from keras import backend

In [46]:
model = Sequential()

# Embedding layer
model.add(
    Embedding(
        input_dim=len(word_idx) + 1,
        output_dim=100,
        weights=None,
        trainable=True))

# Recurrent layer
model.add(
    LSTM(
        64, return_sequences=False, dropout=0.1,
        recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_words, activation='softmax'))

# Compile the model
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         3283900   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32839)             2134535   
Total params: 5,464,835
Trainable params: 5,464,835
Non-trainable params: 0
_________________________________________________________________


For each word the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next word.

## Train the model

In [47]:
h = model.fit(X_train, y_train, epochs = 10, batch_size = 50, 
          verbose = 1)## Train the model

Epoch 1/10
6510/6510 [==============================] - 26s 4ms/step - loss: 8.2421 - acc: 0.0590
Epoch 2/10
6510/6510 [==============================] - 25s 4ms/step - loss: 6.7479 - acc: 0.0619
Epoch 3/10
6510/6510 [==============================] - 24s 4ms/step - loss: 6.4733 - acc: 0.0599
Epoch 4/10
6510/6510 [==============================] - 24s 4ms/step - loss: 6.2822 - acc: 0.0639
Epoch 5/10
6510/6510 [==============================] - 24s 4ms/step - loss: 6.1638 - acc: 0.0636
Epoch 6/10
6510/6510 [==============================] - 24s 4ms/step - loss: 6.0502 - acc: 0.0677
Epoch 7/10
6510/6510 [==============================] - 24s 4ms/step - loss: 5.9419 - acc: 0.0731
Epoch 8/10
6510/6510 [==============================] - 24s 4ms/step - loss: 5.8369 - acc: 0.0773
Epoch 9/10
6510/6510 [==============================] - 24s 4ms/step - loss: 5.7196 - acc: 0.0805
Epoch 10/10
6510/6510 [==============================] - 24s 4ms/step - loss: 5.6256 - acc: 0.0863


### Save Model

In [0]:
# save the model to file
model.save('./data/externallab9.h5')

## If you have already trained the model and saved it, you can load a pretrained model

In [0]:
# load the model
model = load_model('./data/externallab9.h5')

### Note: After loading the model run  model.fit()  to continue training form there, if required.

In [50]:
model.fit(X_train, y_train, batch_size=50, epochs=10)

Epoch 1/10
6510/6510 [==============================] - 24s 4ms/step - loss: 5.5260 - acc: 0.0892
Epoch 2/10
6510/6510 [==============================] - 23s 4ms/step - loss: 5.4303 - acc: 0.0939
Epoch 3/10
6510/6510 [==============================] - 23s 4ms/step - loss: 5.3346 - acc: 0.1017
Epoch 4/10
6510/6510 [==============================] - 23s 4ms/step - loss: 5.2329 - acc: 0.1123
Epoch 5/10
6510/6510 [==============================] - 23s 4ms/step - loss: 5.1190 - acc: 0.1229
Epoch 6/10
6510/6510 [==============================] - 23s 4ms/step - loss: 4.9720 - acc: 0.1347
Epoch 7/10
6510/6510 [==============================] - 23s 4ms/step - loss: 4.8330 - acc: 0.1535
Epoch 8/10
6510/6510 [==============================] - 23s 4ms/step - loss: 4.6820 - acc: 0.1705
Epoch 9/10
6510/6510 [==============================] - 23s 4ms/step - loss: 4.5341 - acc: 0.1839
Epoch 10/10
6510/6510 [==============================] - 23s 4ms/step - loss: 4.4029 - acc: 0.2037


## Evaluation

In [51]:
print(model.evaluate(X_train, y_train, batch_size = 20))
print('\nModel Performance: Log Loss and Accuracy on validation data')
print(model.evaluate(X_valid, y_valid, batch_size = 20))

6510/6510 [==============================] - 7s 1ms/step
[3.9518451298803043, 0.2923195108588207]

Model Performance: Log Loss and Accuracy on validation data
2790/2790 [==============================] - 3s 1ms/step
[6.197138670097543, 0.264516132813628]


## Generate text

In [52]:
seed_length=50
new_words=50
diversity=1
n_gen=1

import random

# Choose a random sequence
seq = random.choice(sequences)

# print seq

# Choose a random starting point
seed_idx = random.randint(0, len(seq) - seed_length - 10)
# Ending index for seed
end_idx = seed_idx + seed_length

gen_list = []

for n in range(n_gen):
    # Extract the seed sequence
    seed = seq[seed_idx:end_idx]
    original_sequence = [idx_word[i] for i in seed]
    generated = seed[:] + ['#']

    # Find the actual entire sequence
    actual = generated[:] + seq[end_idx:end_idx + new_words]
        
    # Keep adding new words
    for i in range(new_words):

        # Make a prediction from the seed
        preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(np.float64)

        # Diversify
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)

        # Softmax
        preds = exp_preds / sum(exp_preds)

        # Choose the next word
        probas = np.random.multinomial(1, preds, 1)[0]

        next_idx = np.argmax(probas)

        # New seed adds on old word
        #             seed = seed[1:] + [next_idx]
        seed += [next_idx]
        generated.append(next_idx)
    # Showing generated and actual abstract
    n = []

    for i in generated:
        n.append(idx_word.get(i, '< --- >'))

    gen_list.append(n)

a = []

for i in actual:
    a.append(idx_word.get(i, '< --- >'))

a = a[seed_length:]

gen_list = [gen[seed_length:seed_length + len(a)] for gen in gen_list]

print('Original Sequence: \n'+' '.join(original_sequence))
print("\n")
# print(gen_list)
print('Generated Sequence: \n'+' '.join(gen_list[0][1:]))
# print(a)

Original Sequence: 
as great a length as i desire for nearly two years i had within a growing burden of bitterness of much of which i have now got rid on the other side of the prison wall there are some poor black soot besmirched trees that are just breaking out into


Generated Sequence: 
edition with cannot is sea seventh impression a balestier etexts fourth who prose study highest these co day every stove 69 impressions of perfection of contents mr minutest ghost is the copy of the http to 'lost david truth of reading proofreading beautiful see the pollajuolo economies' february march him
